In [1]:
from brainlit.utils import session
import napari

In [2]:
url = "s3://mouse-light-viz/precomputed_volumes/brain1"
sess = session.NeuroglancerSession(url=url, url_segments=url+"_segments", mip=0)

In [11]:
import numpy as np
from skimage import draw
import random
from cloudvolume.lib import Bbox
import pandas as pd
from tqdm import tqdm as tqdm

scale = sess.cv_segments.scales[0]["resolution"]
arr = []
sess.cv.progress = False  # don't print download
for seg_id in [2]:  # list of segments
    vertices = sess.cv_segments.skeleton.get(seg_id).vertices
    for i in tqdm(range(len(vertices))):
        v0 = (vertices[i]/scale).astype(int)  # point 0
        v1 = (vertices[i+1]/scale).astype(int)  # point 1
        coords = np.array(draw.line_nd(v0, v1))  # coords of line between points
        inds = random.sample(range(coords.shape[1]), 2)  # get X random points (here, 2)
        for ind in inds:
            coord = coords[:,ind]
            bbox = Bbox(np.subtract(coord,[3,3,3]), np.add(coord,[4,4,4]))  # get box around point
            data = sess.pull_bounds_img(bbox)  # get data
            data_off = sess.pull_bounds_img(bbox+[50,50,50])  # get (50,50,50)-shifted data
            arr.append([1, seg_id, i, ind, data.flatten()])
            arr.append([0, seg_id, i, ind, data_off.flatten()])
header = ["label", "seg id", "vert id", "interp id", "data"]
df = pd.DataFrame(arr, columns=header)

  0%|          | 0/1650 [00:01<?, ?it/s]


ValueError: setting an array element with a sequence.

In [4]:
import _pickle as pkl
pkl.dump(df, open("df.pkl", "wb"))

NameError: name 'df' is not defined

In [5]:
X = np.squeeze(np.array([i for i in df["data"]]))
y = df["label"]

NameError: name 'df' is not defined

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
y_score = clf.predict(X_test)

NameError: name 'X' is not defined

In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('MLP ROC')
plt.legend(loc="lower right")
plt.show()

NameError: name 'y_test' is not defined

In [8]:
from sklearn.linear_model import LogisticRegression
X_center = X[:, 171].reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X_center, y, stratify=y, random_state=1)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_score = clf.predict(X_test)

NameError: name 'X' is not defined

In [9]:
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LogRegression Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

NameError: name 'y_test' is not defined